In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from LPRnet.LPRnet_separable import *
from generator import DataGenerator
import time
PROJECT_NAME = "LPRnet_keras"
MODEL_NAME = "lprnet_straug_twofonts"

IMAGE_SHAPE = [94,24]
CHARS = "ABCDEFGHIJKLMNPQRSTUVWXYZ0123456789" # exclude I, O
CHARS_DICT = {char:i for i, char in enumerate(CHARS)}
DECODE_DICT = {i:char for i, char in enumerate(CHARS)}
NUM_CLASS = len(CHARS)+1

In [7]:
import glob
import cv2

real_images_val = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\valid\\*\\*.png')
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')

data = []
labels = []
val_data = []
val_labels = []

for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace("O","0")
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))/256
    data.append(image)
    labels.append([CHARS_DICT[i] for i in label.split('_')[0]])

for file in real_images_val:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace("O","0")
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))
    val_data.append(image)
    val_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])

training_set = np.array(data,dtype=np.float32)
training_labels = np.array(labels)
ragged = tf.ragged.constant(training_labels).to_tensor()
real_dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).batch(64)


C:\Users\carlos\AppData\Local\Temp/ipykernel_15412/2036217571.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_labels = np.array(labels)


In [2]:
if os.path.exists("trained_models//{}".format(MODEL_NAME)):
    model = keras.models.load_model(
        "trained_models//{}".format(MODEL_NAME), custom_objects={"global_context": global_context, "CTCLoss": CTCLoss  }
    )
    print("loading model")
else:
    model = LPRnet()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),loss =CTCLoss)
    model.build((1,24,94,3))
    print("creating model")

loading model


In [9]:
from gen_plates_keras import *
gen = ImageGenerator()
def generate_dataset(num = 100):
    data, labels = gen.generate_images(num)
    gen_labels = []
    for label in labels:
        gen_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])
    pics =np.array(data)
    labels = np.array(labels)
    training_set = np.array(pics,dtype=np.float32)
    training_labels = np.array(gen_labels)
    ragged = tf.ragged.constant(training_labels).to_tensor()
    dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).shuffle(640).batch(64)
    return dataset
test_dataset = generate_dataset()

C:\Users\carlos\AppData\Local\Temp/ipykernel_15412/2355684483.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_labels = np.array(gen_labels)


In [7]:

TFLITE_PATH = 'tflite_models'
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


with open("./{}/{}.tflite".format(TFLITE_PATH,MODEL_NAME), 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmp_sfbktnx\assets


INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmp_sfbktnx\assets


In [8]:
interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\{}.tflite".format(MODEL_NAME))
#interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\keras_lprnet_separable.tflite")
import numpy as np
import time 
import glob
import cv2
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.resize_tensor_input(input_details[0]['index'], (1, 24, 94, 3))
interpreter.allocate_tensors()
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')
start = time.time()
ctr = 0
batch_images=[]
batch_labels = []
for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    batch_labels.append(label)
    image = cv2.imread(file)
    test_image = cv2.resize(image,(94,24))/256
    test_image = test_image.astype(np.float32)
    batch_images.append(test_image)
    test_image = np.expand_dims(test_image,axis=0)
    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    decoded = keras.backend.ctc_decode(output_data,(24,),greedy=False)
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text:
        print(text, " "+ label)
        ctr += 1
print(time.time()-start)
print(ctr,len(real_images))

DAF9190  DAF9190
DAJ1083  DAJ1083
DCP7079  DCP7079
NBU4828  NBU4828
NCH8029  NCH8029
NFX7563  NFX7563
NGS2592  NGS2592
DAG3746  DAG3746
DAH3150  DAH3150
DAL7407  DAL7407
DCP5431  DCP5431
DCP7323  DCP7323
NA36503  NA36503
NBA4750  NBA4750
NBK6857  NBK6857
NBR9407  NBR9407
NC04583  NC04583
NCD4761  NCD4761
NCM8042  NCM8042
NCM9147  NCM9147
NDE9713  NDE9713
NDF2712  NDF2712
NED3275  NED3275
NGJ4631  NGJ4631
CAB1069  CAB1069
NA02641  NA02641
NBC2309  NBC2309
NDT7127  NDT7127
CDZ5495  CDZ5495
DAL8625  DAL8625
NAT6342  NAT6342
NDC2372  NDC2372
AXA3085  AXA3085
NBE1565  NBE1565
307PZA  307PZA
BAC1219  BAC1219
DAT1060  DAT1060
GCS7606  GCS7606
NAT1827  NAT1827
NCR8656  NCR8656
NEX7507  NEX7507
NFY2439  NFY2439
NFY2439  NFY2439
DAF4129  DAF4129
DAJ4126  DAJ4126
NBV6196  NBV6196
DBA1734  DBA1734
NGY2496  NGY2496
DAC7436  DAC7436
NFW9824  NFW9824
NAE2616  NAE2616
NDX8504  NDX8504
DAH4645  DAH4645
UAC234  UAC234
DAB1391  DAB1391
DAL8038  DAL8038
NAB9128  NAB9128
NCU9737  NCU9737
NEA9103  NEA9103
N

In [9]:
tflite_interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\{}.tflite".format(MODEL_NAME))
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()
tflite_interpreter.resize_tensor_input(input_details[0]['index'], (193, 24, 94, 3))
tflite_interpreter.resize_tensor_input(output_details[0]['index'], (193, 24,36))
tflite_interpreter.allocate_tensors()

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()
print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])
start = time.time()
batch = np.array(batch_images)
tflite_interpreter.set_tensor(input_details[0]['index'], batch)
tflite_interpreter.invoke()
output_data = tflite_interpreter.get_tensor(output_details[0]['index'])
ctr = 0
for k,v in enumerate(output_data):
        text = ""
        print(v.shape)
        undecoded=np.expand_dims(v,axis=0)
        decoded = keras.backend.ctc_decode(undecoded,(24,),greedy=True)
        for j in np.array(decoded[0]).reshape(24):
            if j >-1:
                text += DECODE_DICT[j]
        if batch_labels[k] == text:
            print(text)
            ctr += 1
print(ctr)
print(time.time()-start)
print(ctr,len(real_images))

== Input details ==
name: serving_default_inputs:0
shape: [193  24  94   3]
type: <class 'numpy.float32'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [193  24  36]
type: <class 'numpy.float32'>
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
DAF9190
(24, 36)
DAJ1083
(24, 36)
(24, 36)
DCP7079
(24, 36)
NBU4828
(24, 36)
NCG1221
(24, 36)
NCH8029
(24, 36)
(24, 36)
(24, 36)
NFX7563
(24, 36)
NGS2592
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
CAQ6650
(24, 36)
CPY711
(24, 36)
DAG3746
(24, 36)
DAH3150
(24, 36)
(24, 36)
DAL7407
(24, 36)
(24, 36)
(24, 36)
DCP5431
(24, 36)
DCP7323
(24, 36)
NA36503
(24, 36)
NAE1322
(24, 36)
NAE2008
(24, 36)
(24, 36)
NBA4750
(24, 36)
NBK6857
(24, 36)
NBP3395
(24, 36)
NBR9407
(24, 36)
NC04583
(24, 36)
NCD4761
(24, 36)
NCM8042
(24, 36)
NCM9147
(24, 36)
NDE9713
(24, 36)
NDF2712
(24, 36)
NDJ3111
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
NED3275
(24, 36)
(24, 36)
NGJ4631
(24, 36)
(24, 36)
(24, 36)
(24, 36)
CA

In [19]:
MODEL_PATH = 'trained_models'
TFLITE_PATH = 'tflite_models'
model = keras.models.load_model(
    os.path.join(MODEL_PATH, MODEL_NAME), custom_objects={"global_context": global_context, "CTCLoss": CTCLoss  }
)

In [10]:
import glob
import cv2
real_images_val = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\*\\*\\*.png')
real_new_plates =glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\TFODCourse\\test frames\\autolabelled\\*.png')
ctr = 0
batch = []
for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace('O',str(0))
    image = cv2.imread(file).astype('float32')
    test_image = cv2.resize(image,(94,24))/256
    batch.append(test_image)
    test_image = np.expand_dims(test_image,axis=0)
    preds = model.predict(test_image) 
    decoded = tf.keras.backend.ctc_decode(preds,(24,))
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text or label == text[:-1] or label == text[1:]:
        ctr += 1
    else:
        print(text," "+ label)
print(ctr)
print(len(real_images))

AXA302B  AXA3028
DAC581B  DAC5818
DD23624AA  DD23624
HQNXT20AA  F0W128
NE0141  NED1441
NG8Z516A  NGB2516
P1E5ZAAA  PIE527
ATF2TTMAA  TIS271
UDSL791A  USL794
WCP28TA  WCP291
KWC079TA  WCQ791
MUR28P3AA  WQK
YZ0G54  YZ9654
ZMM74BA  ZMM748
RPV720TAA  ZPV201
LB02158LA  DU2156
AACGB84  AAC6884
AAN88T4  AAN8814
ABZL4748A  ABL4748
ABT3G0B  ABT3609
AHA320B  AHA3208
AJA4T35  AJA4135
AJA4T40A  AJA4140
DAB4533AA  DAB4533
AH5784  DAH5784
0AN5585  DAN558B
LDAN98B50  DAN9650
DMM572AA  DMW572
EH5822A  EH5022
MV3783AA  MV3783
4WCJ4567  NCJ4567
NCCV535A  NDQ536
LNKT2586A  NK1258
6NK1258XA  NK1258
N0855QA  N08559
P89ZT07AA  P4Z107
PFJ878UA  PFJ870
RJNHBT3JVAA  RJN813
RN01GEHL  RND688
SJYXX799A  SJX799
TGGBPG  TGG996
TYNV552  TYW552
UAE305A  UAE505
UBK95TA  UBK951
UHR83TA  UHR831
UHYST5AA  UHY915
ULB5G2  ULB562
LPH549A  UPH549
VCEZ00A  VCE200
VCE2GQA  VCE200
RVDUMFZA  VDU792
MHJB22QA  WJB229
R4GML12ZJ9VA  XRC239
XTTB8BAA  XTT888
Z4KNUXT5JAA  ZKU755
TZRU36TAA  ZRU361
AAA8067  AAA9067
AAH8437  AAM8437
AAN3G

In [24]:
val_training_set = np.array(val_data,dtype=np.float32)
val_training_labels = np.array(val_labels)
val_ragged = tf.ragged.constant(val_training_labels).to_tensor()
val_dataset = tf.data.Dataset.from_tensor_slices((val_training_set,val_ragged)).shuffle(640).batch(64)

C:\Users\carlos\AppData\Local\Temp/ipykernel_21744/3660101917.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val_training_labels = np.array(val_labels)


In [34]:
import glob
images =  glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\*\\*\\*.png')
print(len(images))

def data_gen():
    a = []
    for i in range(500):
        filename = images[i]
        img = cv2.imread(filename)
        img = cv2.resize(img, (94,24))
        img = img / 255.0
        img = img.astype('float32')
        a.append(img)
    a = np.array(a)
    print(a.shape) # a is np array of 160 3D images
    img = tf.data.Dataset.from_tensor_slices(a).batch(1)
    for i in img.take(32):
        print(i)
        yield [i]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
if True:
    # converter.optimizations = [tf.lite.Optimize.DEFAULT]
    # converter.representative_dataset = representative_data_gen
    # converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    # converter.inference_input_type = tf.uint8
    # converter.inference_output_type = tf.uint8
    converter.representative_dataset = data_gen
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8, tf.lite.OpsSet.SELECT_TF_OPS]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    converter.allow_custom_ops = True

    print(converter.experimental_new_quantizer)  # outputs True
    print(converter.experimental_new_converter)  # outputs True
else:
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
    converter.optimizations = []

tflite_model = converter.convert()

with open('quant.tflite', 'wb') as f:
    f.write(tflite_model)

786
True
True


INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmp94kgnn4s\assets


INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmp94kgnn4s\assets


(500, 24, 94, 3)
tf.Tensor(
[[[[0.1254902  0.14117648 0.1764706 ]
   [0.1254902  0.13725491 0.18039216]
   [0.11764706 0.14901961 0.22352941]
   ...
   [0.14117648 0.6901961  0.8352941 ]
   [0.         0.29411766 0.40784314]
   [0.04313726 0.12941177 0.21568628]]

  [[0.11372549 0.14117648 0.18431373]
   [0.11764706 0.14509805 0.1882353 ]
   [0.11764706 0.14901961 0.21176471]
   ...
   [0.20392157 0.59607846 0.7294118 ]
   [0.05490196 0.3019608  0.41568628]
   [0.18431373 0.27058825 0.35686275]]

  [[0.23921569 0.26666668 0.31764707]
   [0.23921569 0.26666668 0.31764707]
   [0.22745098 0.2627451  0.3254902 ]
   ...
   [0.14117648 0.32156864 0.44705883]
   [0.1254902  0.29411766 0.39607844]
   [0.18039216 0.29803923 0.39215687]]

  ...

  [[0.6039216  0.6666667  0.77254903]
   [0.627451   0.73333335 0.89411765]
   [0.4117647  0.6509804  0.8509804 ]
   ...
   [0.5686275  0.627451   0.9529412 ]
   [0.6627451  0.68235296 0.9529412 ]
   [0.6313726  0.5568628  0.73333335]]

  [[0.6392157  0.

tf.Tensor(
[[[[0.21176471 0.22745098 0.23529412]
   [0.21176471 0.22745098 0.23529412]
   [0.21176471 0.22745098 0.23529412]
   ...
   [0.04313726 0.04313726 0.04313726]
   [0.03529412 0.03529412 0.03529412]
   [0.03137255 0.03137255 0.03137255]]

  [[0.21568628 0.23137255 0.23921569]
   [0.21176471 0.22745098 0.23529412]
   [0.21176471 0.22745098 0.23529412]
   ...
   [0.03921569 0.03921569 0.03921569]
   [0.03137255 0.03137255 0.03137255]
   [0.02352941 0.02352941 0.02352941]]

  [[0.22352941 0.23137255 0.23921569]
   [0.21960784 0.22745098 0.23529412]
   [0.21960784 0.22745098 0.23529412]
   ...
   [0.04313726 0.04313726 0.04313726]
   [0.03137255 0.03137255 0.03137255]
   [0.01960784 0.01960784 0.01960784]]

  ...

  [[0.38039216 0.3647059  0.3764706 ]
   [0.39215687 0.3764706  0.3882353 ]
   [0.40392157 0.3882353  0.4       ]
   ...
   [0.01176471 0.01176471 0.01176471]
   [0.01176471 0.01176471 0.01176471]
   [0.01176471 0.01176471 0.01176471]]

  [[0.23921569 0.23529412 0.243137